In [ ]:
from pathlib import Path
import numpy as np
import cartopy.crs as ccrs
import rasterio
import matplotlib.pyplot as plt


DATA_DIR = Path('./data')
LANDSAT_FILENAMES = [
    'LC08_L1GT_217105_20171203_20171207_01_T2_B4.TIF',
    'LC08_L1GT_217105_20171203_20171207_01_T2_B3.TIF',
    'LC08_L1GT_217105_20171203_20171207_01_T2_B2.TIF',
]
SUBSAMPLE = 5

rgb = []
for filename in LANDSAT_FILENAMES:
    with rasterio.open(DATA_DIR / filename, 'r') as src:
        # read image into ndarray
        im = src.read(1).astype(float)

        # Set 0 values (no data) to nan
        im[im == 0] = np.nan

        # subsampling
        im = im[::SUBSAMPLE, ::SUBSAMPLE]

        rgb.append(im)

        # redundant since should be the same for each
        transform = src.transform
        width, height = src.width, src.height

# move first axis to last
rgb = np.moveaxis(rgb, 0, -1)

# mask zero values (no data)
mask = np.isnan(rgb)
rgb = np.ma.masked_where(mask, rgb)

# normalize
rgb = rgb / rgb.max()

# use mask as transparency array
flat_mask = np.logical_not(np.expand_dims(mask.any(axis=-1), axis=-1))
rgba = np.dstack([rgb, flat_mask])

# calculate extent of raster
xmin = transform[2]
xmax = transform[2] + transform[0] * width
ymin = transform[5] + transform[4] * height
ymax = transform[5]

# define cartopy crs for the raster, based on rasterio metadata
src_crs = ccrs.SouthPolarStereo(true_scale_latitude=-71)

# create figure
ax = plt.axes(projection=src_crs)

# plot raster
ax.imshow(
    rgba,
    origin='upper',
    extent=[xmin, xmax, ymin, ymax],
    transform=src_crs,
    interpolation='nearest',
)

# plot coastlines
ax.coastlines(resolution='10m')

plt.show()